In [1]:
import pandas as pd
import pprint
import math
pp = pprint.PrettyPrinter(indent = 4)

#read from csv (TODO change this file to updated version)
df = pd.read_csv('./sequence_categoryok.csv')

columnlist = df.columns.tolist()


In [2]:
#delete rows with no tasks
xy = df[df.Tasks.notnull()]


#fill in Category, questions, field according to row above
xy['Category'].fillna(method='ffill', inplace=True)
xy['Questions'].fillna(method='ffill', inplace=True)   
xy['Field'].fillna(method='ffill', inplace=True) 


#reset index to contingent numbers after deleting rows with no tasks
xy.reset_index(drop=True, inplace=True)

/Users/emily/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [3]:
#record sequence of tasks
# group by (Category, Questions Field). If any one value changes, start new sequence from 1
index = [] #stores all sequence
i = 1
curr_category = xy['Category'][0]
curr_question = xy['Questions'][0]
curr_field = xy['Field'][0]

for j in range(len(xy)):
    c = xy['Category'][j]
    q = xy['Questions'][j]
    f = xy['Field'][j]
    
    if c != curr_category:
        i = 1
        curr_category = c
    if q != curr_question:
        i = 1
        curr_question = q
    if f != curr_field:
        i = 1
        curr_field = f
   
    index.append(i)
    i += 1
    
xy['Sequence'] = index #add sequence as a new column to visulize
xy

/Users/emily/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Category,Questions,Field,Tasks,Assignment,Visibility,Time,Min Time FORMULA,Budget,Sequence
0,General,Wedding (Event) Date,text,Discuss wedding weekend events with client,Planner,Yes,7 months,5,NaN,1
1,General,Wedding (Event) Date,text,Register for gifts,Client,Yes,7 months,5,NaN,2
2,General,Wedding (Event) Date,text,- Obtain your marriage license,Client,Yes,3 months,1,NaN,3
3,General,Wedding (Event) Date,text,"Purchase gifts for wedding party, parents and ...",Client,Yes,2 months,1,NaN,4
4,General,Guest Count,text,Compile Guest list,Client,Yes,9 months,6,NaN,1
5,General,Guest Count,text,- Collect Guest email and physical addresses,Client,Yes,8 months,6,NaN,2
6,General,Guest Count,text,- RSVP deadline,Planner,Yes,1 Month,1,NaN,3
7,General,Guest Count,text,- Send Final guest count to Planner,Client,Yes,1 month,2 weeks,NaN,4
8,General,Guest Count,text,- Update floor plan based on final guest count...,Planner,No,4 weeks,4 weeks,NaN,5
9,General,Guest Count,text,Final table/seat assignments due,Client,Yes,1 month,3 weeks,NaN,6


In [4]:
#assign columns to lists
category = [x for x in xy['Category']]
question = [x for x in xy['Questions']]
field = [x for x in xy['Field']]
description = xy['Tasks']
assignment = [x for x in xy['Assignment']]
visibility = [x for x in xy['Visibility']]
timing = [x for x in xy['Time']]
mintime = [x for x in xy['Min Time FORMULA']]

In [5]:
for i in timing:
    print(i)
    print(type(i))

7 months
<class 'str'>
7 months
<class 'str'>
3 months
<class 'str'>
2 months
<class 'str'>
9 months
<class 'str'>
8 months
<class 'str'>
1 Month
<class 'str'>
1 month
<class 'str'>
4 weeks
<class 'str'>
1 month
<class 'str'>
3 weeks
<class 'str'>
3 weeks
<class 'str'>
2 weeks
<class 'str'>
10 months
<class 'str'>
10 month
<class 'str'>
9 months
<class 'str'>
9 months
<class 'str'>
9 months
<class 'str'>
9 months
<class 'str'>
4 months
<class 'str'>
10 months
<class 'str'>
8 months
<class 'str'>
7 months
<class 'str'>
4 months
<class 'str'>
4 months
<class 'str'>
6 months
<class 'str'>
5 months
<class 'str'>
8 months
<class 'str'>
7 months
<class 'str'>
7 months
<class 'str'>
5 months
<class 'str'>
5 months
<class 'str'>
8 months
<class 'str'>
4 months
<class 'str'>
6 months
<class 'str'>
4 months
<class 'str'>
2 months
<class 'str'>
1 month
<class 'str'>
7 months
<class 'str'>
2 months
<class 'str'>
10 months
<class 'str'>
10 months
<class 'str'>
10 months
<class 'str'>
10 months
<cla

In [14]:
#get rid of "-" in front of description
description = [x.replace("- ", "", 1) for x in description]
#get rid of " " after description
description = [x.strip() for x in description]


timing_new = []
mintime_new = []
#timing and mintime 
for i in range(len(timing)):
    
    #if Null value, keep as null
    if isinstance(timing[i], float) or isinstance(mintime[i], float):
        timing_new.append(None)
        mintime_new.append(None)
        continue
    
    
    #detect number for timing and mintime
    timing_num = int(''.join(filter(str.isdigit, timing[i])))
    mintime_num = int(''.join(filter(str.isdigit, mintime[i])))
    
    #detect starting alphabet (unit) for timing and mintime
    unit_timing= ''.join(filter(lambda x: x.isalpha(), timing[i]))[0]
    mintime_alpha = ''.join(filter(lambda x: x.isalpha(), mintime[i]))

    #if no units specified for mintime, default MONTH
    if mintime_alpha == '' or mintime_alpha[0] == 'm':
        unit_mintime = 'm'
    else:
        unit_mintime = mintime_alpha[0]
    
    #change all units to DAYS
    if unit_timing == 'm':
        timing_new.append(timing_num * 30)
    elif unit_timing == 'w':
        timing_new.append(timing_num * 7)
    else:
        timing_new.append(timing_num)
        
    if unit_mintime == 'm':
        mintime_new.append(mintime_num * 30)
    elif unit_mintime == 'w':
        mintime_new.append(mintime_num * 7)
    else:
        mintime_new.append(mintime_num)
        
    timing_new[i] *= -1
    mintime_new[i] *= -1

            
print(timing_new)
print(mintime_new)

timingratio = [-x/300 for x in timing_new]
        
#visibility
# for i, v in enumerate(visibility):
#     if v != 'yes':
#         visibility[i] = 'no'

[-210, -210, -90, -60, -270, -240, -1, -30, -28, -30, -21, -21, -14, -300, -300, -270, -270, -270, -270, -120, -300, -240, -210, -120, -120, -180, -150, -240, -210, -210, -150, -150, -240, -120, -180, -120, -60, -30, -210, -60, -300, -300, -300, -300, -300, -300, -300, -90, -120, -300, -300, -300, -300, -300, -270, -90, -120, -300, -300, None, -90, -60, -240, -120, -120, -120, -60, -210, -120, -120, -60, -180, -270, -270, -270, -150, -90, -90, -30, -21, -150, -90, -90, -30, -21, -90, -90, -120, None, -90, -120, -90, -180, -210, -120, -90, -150, None, -150, -150, -270, -270, -240, -90, -90, -30, -42, -30, -42, -30, -42, -300, -270, -270, -30, -270, -30, -240, -240, -240, -240, None, -210, -210, -240, -240, -210, -240, -2250, -210, -180, -210, -210, -180, -180, -150, -120, -120, -120, -750, -120, -120, -90, -63, -90, -60, -30, -28, -4, -210, -180, -150, -180, -180, -180, -180, -150, -180, -180, -150, -180, -180, -150, -270, -270, -270, -90, -210, -1950, -3, -210, -1950, -3, -150, -150, -

In [15]:
xy["newtime"] = timing_new
xy["newmin"] = mintime_new

xy

/Users/emily/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/emily/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Category,Questions,Field,Tasks,Assignment,Visibility,Time,Min Time FORMULA,Budget,Sequence,newtime,newmin
0,General,Wedding (Event) Date,text,Discuss wedding weekend events with client,Planner,Yes,7 months,5,NaN,1,-210.0,-150.0
1,General,Wedding (Event) Date,text,Register for gifts,Client,Yes,7 months,5,NaN,2,-210.0,-150.0
2,General,Wedding (Event) Date,text,- Obtain your marriage license,Client,Yes,3 months,1,NaN,3,-90.0,-30.0
3,General,Wedding (Event) Date,text,"Purchase gifts for wedding party, parents and ...",Client,Yes,2 months,1,NaN,4,-60.0,-30.0
4,General,Guest Count,text,Compile Guest list,Client,Yes,9 months,6,NaN,1,-270.0,-180.0
5,General,Guest Count,text,- Collect Guest email and physical addresses,Client,Yes,8 months,6,NaN,2,-240.0,-180.0
6,General,Guest Count,text,- RSVP deadline,Planner,Yes,1 Month,1,NaN,3,-1.0,-30.0
7,General,Guest Count,text,- Send Final guest count to Planner,Client,Yes,1 month,2 weeks,NaN,4,-30.0,-14.0
8,General,Guest Count,text,- Update floor plan based on final guest count...,Planner,No,4 weeks,4 weeks,NaN,5,-28.0,-28.0
9,General,Guest Count,text,Final table/seat assignments due,Client,Yes,1 month,3 weeks,NaN,6,-30.0,-21.0


In [ ]:

  
# Calling DataFrame constructor after zipping 
# both lists, with columns specified 
df = pd.DataFrame(list(zip(description, question, field, timing, timingratio, mintime, assignment, client, category)), 
               columns =['description', 'question', 'field', 'timing', 'timingratio', 'mintime', 'assignment', 'client', 'category']) 
df 




In [ ]:
#writing results to sample.csv
content = df.to_csv()
with open('sample.csv','w') as file:
    print("writing")
    file.write(content)
    file.close()
